In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Prepare Model
## Load pre-trained model

In [10]:
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-07-23 07:50:35--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.126.128, 74.125.70.128, 74.125.132.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   237MB/s    in 0.4s    

2023-07-23 07:50:35 (237 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [11]:
pretrained_model = InceptionV3(input_shape=(150, 150, 3),
                               include_top=False,
                               weights=None)

weight_file = "/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
pretrained_model.load_weights(weight_file)

for layer in pretrained_model.layers:
    layer.trainable = False

In [12]:
pretrained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 74, 74, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 74, 74, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                       

select mixed7 layer (somewhere in between -> not too specialized)

In [13]:
last_layer = pretrained_model.get_layer('mixed7')
last_output = last_layer.output

## Stack layers on top

In [14]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model
from tensorflow.keras import layers

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pretrained_model.input, x)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 74, 74, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 74, 74, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                              

In [22]:
model.compile(optimizer=RMSprop(learning_rate=0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

# Prepare Data

In [16]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

--2023-07-23 07:52:44--  https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.126.128, 74.125.70.128, 74.125.132.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   197MB/s    in 0.3s    

2023-07-23 07:52:44 (197 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [17]:

import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Extract the archive
zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

# Define our example directories and files
base_dir = 'tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats') 
train_dogs_dir = os.path.join(train_dir, 'dogs') 

validation_cats_dir = os.path.join(validation_dir, 'cats') 
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [23]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 32,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 32,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Train

In [24]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
#             steps_per_epoch = 100,
            epochs = 20,
#             validation_steps = 50,
            verbose = 2)

Epoch 1/20
63/63 - 20s - loss: 0.1927 - accuracy: 0.9240 - val_loss: 0.0935 - val_accuracy: 0.9690 - 20s/epoch - 312ms/step
Epoch 2/20
63/63 - 16s - loss: 0.1812 - accuracy: 0.9310 - val_loss: 0.0810 - val_accuracy: 0.9750 - 16s/epoch - 246ms/step
Epoch 3/20
63/63 - 15s - loss: 0.1629 - accuracy: 0.9325 - val_loss: 0.0834 - val_accuracy: 0.9680 - 15s/epoch - 236ms/step
Epoch 4/20
63/63 - 16s - loss: 0.1552 - accuracy: 0.9310 - val_loss: 0.1110 - val_accuracy: 0.9570 - 16s/epoch - 248ms/step
Epoch 5/20
63/63 - 16s - loss: 0.1468 - accuracy: 0.9395 - val_loss: 0.0994 - val_accuracy: 0.9610 - 16s/epoch - 249ms/step
Epoch 6/20
63/63 - 15s - loss: 0.1309 - accuracy: 0.9485 - val_loss: 0.1639 - val_accuracy: 0.9490 - 15s/epoch - 237ms/step
Epoch 7/20
63/63 - 16s - loss: 0.1609 - accuracy: 0.9345 - val_loss: 0.0785 - val_accuracy: 0.9710 - 16s/epoch - 249ms/step
Epoch 8/20
63/63 - 15s - loss: 0.1365 - accuracy: 0.9455 - val_loss: 0.1297 - val_accuracy: 0.9430 - 15s/epoch - 234ms/step
Epoch 9/